### Àlex Correa Orri 1564967

#### Dataset: https://www.kaggle.com/shaurov/website-classification-using-url

In [30]:
reset()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  (), please run `%reset?` for details


### MÒDULS I LLIBRERIES

In [31]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats
import os
import seaborn as sns
from scipy.stats import shapiro, spearmanr
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import ipympl
from sklearn.linear_model import LogisticRegression
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib widget

### DIRECTORI DE TREBALL

In [32]:
os.getcwd()
path = 'C:/Users/alexc/OneDrive/Escritorio/UNI/3er/Aprenentatge Computacional/PRAC_KAGGLE'
os.chdir(path)
os.getcwd()
# Visualitzarem només 3 decimals per mostra
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

'C:\\Users\\alexc\\OneDrive\\Escritorio\\UNI\\3er\\Aprenentatge Computacional\\PRAC_KAGGLE'

### LECTURA DE DADES

In [33]:
# Carreguem dataset d'exemple
dataset = pd.read_csv('URL_Classification.csv', names=['n', 'url', 'class'], delimiter=',')

In [34]:
#dataset

### MODIFICACIÓ DEL DATASET

In [35]:
dataset = dataset.drop(['n'], axis=1)

In [36]:
print(dataset['class'].unique())
len(dataset['class'].unique())

['Adult' 'Arts' 'Business' 'Computers' 'Games' 'Health' 'Home' 'Kids'
 'News' 'Recreation' 'Reference' 'Science' 'Shopping' 'Society' 'Sports']


15

#### Per a fer proves reduim el sample a 10000 dades

In [37]:
s=10000
dataset1 = dataset.sample(n=s, random_state=10)

In [38]:
print(dataset1['class'].unique())
len(dataset1['class'].unique())

['Games' 'Reference' 'Society' 'Computers' 'Arts' 'Kids' 'Business'
 'Shopping' 'Recreation' 'Health' 'Sports' 'Science' 'Home' 'News' 'Adult']


15

In [39]:
#dataset1

#### Consultem quines són les paraules més comunes als url del dataset

In [40]:
#dataset1

In [41]:
urls=''
for url in dataset1['url']:
    urls += url
#urls: text concatenat amb tots els strings (sense espais)

In [42]:
vect = CountVectorizer()
mat = vect.fit_transform([urls])
mat
counts = pd.DataFrame(mat.toarray(), columns=vect.get_feature_names())
counts #pandas dataframe amb els valors de les paraules més comunes al string

,00,00004fhttp,0002786,001,00100,0011,0020,0029229http,007aufgcn,00_52,...,zope,zorrofx,zsanta,zsp,zulem,zulu,zumpano,zynewave,zytco,zzpat
0,8,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [43]:
sort_counts = counts.sort_values(by =0, axis=1)
sort_counts = sort_counts[sort_counts.columns[::-1]]

In [44]:
sort_counts #dataframe ordenat per la paraula més comuna

,www,http,com,comhttp,org,htmlhttp,htmhttp,edu,co,uk,...,nosa,norzagaray,norwich,norwalkctlawfirm,northwestsoftball,northwestpatentlaw,northwest,northtexas,northstarrocketry,juniorcorinthians
0,8061,5494,5012,1267,1134,1086,690,519,446,429,...,1,1,1,1,1,1,1,1,1,1


In [45]:
sort_counts.iloc[:, : 50].columns

Index(['www', 'http', 'com', 'comhttp', 'org', 'htmlhttp', 'htmhttp', 'edu',
       'co', 'uk', 'net', 'geocities', 'index', 'amp', 'orghttp', 'tripod',
       'angelfire', 'ca', 'au', 'gov', 'ac', 'ukhttp', 'us', 'members',
       'nethttp', 'home', 'de', 'id', 'newadvent', 'cathen', 'php', 'en',
       'shtmlhttp', 'nl', 'reviews', 'imdb', 'cgi', 'asp', 'yahoo', 'groups',
       'html', 'news', 'free', 'group', 'users', 'sports', 'wiki', 'title',
       'asphttp', 'wikipedia'],
      dtype='object')

In [46]:
sort_counts_del = sort_counts
sort_counts_del = sort_counts_del.drop(['www', 'http', 'com', 'comhttp', 'org', 'htmlhttp', 'htmhttp', 'edu', 'co', 'uk', 'net', 'orghttp','html','nethttp','shtmlhttp', 'ukhttp'], axis=1)

In [47]:
sort_counts_del.iloc[:, : 150]

,geocities,index,amp,tripod,angelfire,ca,au,gov,ac,us,...,titles,people,tech,03,pubs,nzhttp,or,content,cmu,bartleby
0,350,315,176,144,134,133,109,107,94,89,...,13,13,13,13,13,13,13,13,13,13


#### Utilitzarem les 100 paraules més comunes (treient els www, http, com, org,...)

In [48]:
words_def = list(sort_counts_del.iloc[:, : 100].columns)
#words_def
for w in words_def:
    if ('http' in w):
        words_def.remove(w)

In [49]:
len(words_def)

95

In [50]:
url_vect = list(dataset1['url'])
vectorizer = CountVectorizer()
fit_vect = vectorizer.fit_transform(url_vect)

dataset1_word = pd.DataFrame(fit_vect.toarray(), columns=vectorizer.get_feature_names())

In [51]:
dataset1_word = dataset1_word[words_def] #matriu (dataset) amb les paraules més comunes

In [52]:
dataset1_word.insert(0, 'url', url_vect)
dataset1_word.insert(1, 'class', list(dataset1['class']))

In [53]:
dataset1_word

,url,class,geocities,index,amp,tripod,angelfire,ca,au,gov,...,eti,beschrijving,soorten,menuentry,uva,selected,math,archive,se,football
0,http://www.aaagames.com/pokemon.htm,Games,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,http://www.azdyslexia.com/,Reference,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,http://www.blondetiger.net/,Society,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,http://www.willoughbywebdesign.com/,Computers,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,http://www.ashevillebarbershop.com/,Arts,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,http://londonphotography.biz,Business,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,http://project-ufo.com,Society,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,http://www.gamespot.com/ps2/rpg/makaikingdomch...,Games,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,http://www.geocities.com/hollywood/academy/3420/,Arts,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
